<left><img width=25% src="img/gw_monogram_2c.png"></left>

# Lecture 6: Introduction to Graph Neural Networks

### CS4907/CS6365 Machine Learning

__Sardar Hamidian__<br>The George Washington Universiry

__Armin Mehrabian__<br>The George Washington Universiry

# Recap: Shallow Encoders

## Limitations of shallow embedding methods:


- **$O(|V|)$ parameters are needed**:
    - No sharing of parameters between nodes
    - Every node has its own unique embedding

- **Inherently "transductive"**:
    - Cannot generate embeddings for nodes that are not seen during training

- **Do not incorporate node features**:
    - Many graphs have features that we can and should leverage

# Deep Graph Encoders

- **Today**: We will now discuss deep methods based on graph neural networks (GNNs):

$$ \text{ENC}(v) = \text{multiple layers of non-linear transformations based on graph structure} $$

- **Note**: All these deep encoders can be combined with node similarity functions defined in Lecture 4.


# Deep Graph Encoders

- **Deep graph encoders** utilize graph neural networks (GNNs) to learn node embeddings by performing multiple layers of non-linear transformations based on the graph structure.

![Deep Graph Encoders Image](./img/deep_graph_encoders.png)



- **Key Advantages**:
    - Can generalize to unseen nodes (inductive)
    - Share parameters across the graph, reducing complexity
    - Can incorporate node features and graph structure effectively


- **Applications**:
    - Node classification
    - Link prediction
    - Community Detection
    - Graph-level tasks (e.g., molecular graph classification)

# Agenda
# Local Network Neighborhoods

- Describe aggregation strategies
- Define computation graphs

# Stacking Multiple Layers

- Describe the model, parameters, and training
- How to fit the model
- Simple example for unsupervised and supervised training


# Setup

- **Assume we have a graph $G$:**
    - $V$ is the **vertex set**
    - $A$ is the **adjacency matrix** (assume binary)
    - $X \in \mathbb{R}^{m \times |V|}$ is a matrix of **node features**
    - $v$: a node in $V$; $N(v)$: the set of neighbors of $v$

- **Node features**:
    - **Social networks**: User profile, User image
    - **Biological networks**: Gene expression profiles, gene functional information
    - **When there is no node feature in the graph dataset**:
        - Indicator vectors (one-hot encoding of a node)
        - Vector of constant 1: $[1, 1, 1, ..., 1]$


# A Naïve Approach

- **Join adjacency matrix and features**
- **Feed them into a deep neural net**:

![Naïve Approach Image](./img/gnn_to_dnn.png)

### Issues with this idea:

- **$O(|V|)$ parameters**
- **Not applicable to graphs of different sizes**
- **Sensitive to node ordering**


# Inspiration from CNNs

![CNN Analogy](./img/cnn_analogy.png)

- Goal is to generalize convolutions beyond simple lattices Leverage node features/attributes (e.g., text, images)


# However in Graph World

![Sliding Window in Graphs](./img/graph_sliding_window.png)

- There is no fixed notion of locality or sliding window on the graph
- Graph is permutation invariant


# From Images to Graphs

- **Single Convolutional Neural Network (CNN) layer with 3x3 filter:**

![CNNs vs Graphs](./img/cnn_vs_graphs.png)

### Idea: Transform information at the neighbors and combine it:
- Transform "messages" $h_i$ from neighbors: $W_i h_i$
- Add them up: $\sum_i W_i h_i$


# Graph Convolutional Networks

### Idea: Node's neighborhood defines a computation graph

- **Determine node computation graph** 
- **Propagate and transform information** 
![Graph Convolutional Networks Image](./img/propagate_and_aggregate.png)

### Learn how to propagate information across the graph to compute node features.


# Idea: Aggregate Neighbors

- Key idea: Generate node embeddings based on local network neighborhoods

![GNN Idea 1](./img/gnn_idea_1.png)


- Intuition: Network neighborhood defines a computation graph
- Every node defines a computa2on graph based on its neighborhood!

![GNN Idea 3](./img/gnn_idea_3.png)


# Deep Model: Many Layers

- Model can be of arbitrary depth:
    - Nodes have embeddings at each layer
    - Layer-0 embedding of node $𝑣$ is its input feature, $𝑥_𝑣$
    - Layer-𝑘 embedding gets information from nodes that are $𝑘$ hops away
    
![GNN Idea 4](./img/gnn_idea_4.png)


# Neighborhood Aggregation

- **Neighborhood aggregation:** Key distinctions are in how different approaches aggregate information across the layers

![GNN Idea 5](./img/gnn_idea_5.png)


# Neighborhood Aggregation

- **Basic approach:** Average information from neighbors and apply a neural network

![GNN Idea 6](./img/gnn_idea_6.png)


# The Math: Deep Encoder

- **Basic approach**: Average neighbor messages and apply a neural network

$$ h_v^0 = x_v $$

$$ h_v^{(l+1)} = \sigma\left(W_l \sum_{u \in N(v)} \frac{h_u^{(l)}}{|N(v)|} + B_l h_v^{(l)}\right), \forall l \in \{0, \dots, L-1\} $$

$$ z_v = h_v^{(L)} $$


# Neighborhood Aggregation

- **Basic approach:** Average information from neighbors and apply a neural network

![GNN Idea 7](./img/gnn_idea_7.png)


# The Math: Deep Encoder


- **Basic approach**: Average neighbor messages and apply a neural network

$$ h_v^{(0)} = x_v $$

$$ h_v^{(k+1)} = \sigma\left(W_k \sum_{u \in N(v)} \frac{h_u^{(k)}}{|N(v)|} + B_k h_v^{(k)}\right), \forall k \in \{0, \dots, K-1\} $$

$$ z_v = h_v^{(K)} $$

- **$W_k$ and $B_k$** are the trainable weight matrices (i.e., what we learn).
- **$z_v$** is the final node embedding.



# Matrix Formulation (1)

- **Many aggregations can be performed efficiently by (sparse) matrix operations:**

$$ H^{(l)} = \left[ h_1^{(l)} \, \dots \, h_{|V|}^{(l)} \right]^T $$

- **What is $H^{(l)}$?**
  - $H^{(l)}$ is a matrix of node embeddings at layer $l$. 
  - Each row $h_v^{(l)}$ represents the hidden embedding (or feature vector) of node $v$ at layer $l$.
  - The dimensions of $H^{(l)}$ are $|V| \times d$, where $|V|$ is the number of nodes and $d$ is the embedding dimension.
  - These embeddings capture information about the node and its neighbors.

- Then: 

$$ \sum_{u \in N(v)} h_u^{(l)} = A_{v,:} H^{(l)} $$

- **What is $D$?**
  - $D$ is a diagonal degree matrix where the diagonal entry $D_{v,v}$ is the degree of node $v$, i.e., $\text{Deg}(v) = |N(v)|$.
  - The inverse of $D$, $D^{-1}$, is also diagonal, where $D^{-1}_{v,v} = \frac{1}{|N(v)|}$.
  - This matrix normalizes the node embeddings by scaling contributions from neighbors based on node degree.


- **Therefore:**

$$ \sum_{u \in N(v)} \frac{h_u^{(l-1)}}{|N(v)|} \longrightarrow H^{(l+1)} = D^{-1} A H^{(l)} $$

- This formula expresses how node embeddings from neighbors are aggregated using sparse matrix multiplication with the adjacency matrix $A$ and normalized by $D^{-1}$.


# Matrix Formulation (2)

- **Re-writing the update function in matrix form:**

$$ H^{(l+1)} = \sigma\left(\tilde{A} H^{(l)} W_l^T + H^{(l)} B_l^T \right) $$


  where:

$$ \tilde{A} = D^{-1} A $$

- **Explanation**:
  - **Neighborhood aggregation**: The term $( \tilde{A} H^{(l)} W_l^T $) aggregates information from the neighbors using the normalized adjacency matrix $( \tilde{A} )$.
  - **Self-transformation**: The term $( H^{(l)} B_l^T $) applies a transformation to the node itself without aggregation from its neighbors.


- **In practice**, this implies that efficient sparse matrix multiplication can be used (since $( \tilde{A} $) is sparse).

- **Note**: Not all GNNs can be expressed in matrix form, especially when the aggregation function is more complex.


# How to train a GNN

- **Node embedding $z_v$** is a function of the input graph.

- **Supervised setting**: we want to minimize the loss $\mathcal{L}$:

$$ \min_{\Theta} \mathcal{L}(y, f(z_v)) $$

  - $y$: node label
  - $\mathcal{L}$ could be L2 if $y$ is a real number, or cross-entropy if $y$ is categorical.

- **Unsupervised setting**:
  - No node label available.
  - Use the graph structure as the supervision!


# Unsupervised Training

### "Similar" nodes have similar embeddings

$$ \mathcal{L} = \sum_{z_u, z_v} \text{CE}(y_{u,v}, \text{DEC}(z_u, z_v)) $$

- Where $y_{u,v} = 1$ when node $u$ and $v$ are **similar**.
- **CE** is the cross entropy.
- **DEC** is the decoder, such as inner product (lecture 4).

- **Node similarity** can be anything from lecture 3, e.g., a loss based on:
  - **Random walks** (node2vec, DeepWalk, struc2vec).
  - **Matrix factorization**.
  - **Node proximity** in the graph.


# Supervised Training

- **Directly train** the model for a supervised task (e.g., **node classification**).
- Use cross-entropy loss (slide 16):

$$ \mathcal{L} = \sum_{v \in V} \left[ y_v \log(\sigma(z_v^T \Theta)) + (1 - y_v) \log(1 - \sigma(z_v^T \Theta)) \right] $$

- **Explanation**:
  - $z_v$: Encoder output, i.e., the node embedding for node $v$.
  - $y_v$: The node class label (1 if the node belongs to a certain class, 0 otherwise).
  - $\Theta$: Classification weights applied to the node embedding.
  - $\sigma$: Sigmoid function, producing the probability that the node belongs to the class.
  
- Example: Safe or toxic drug? The node embedding represents a drug, and the classification determines the outcome (safe/toxic).


# Training Overview

![GNN Design Overview 1 2](./img/gnn_design_1_2.png)


# Training Overview

![GNN Design Overview 3](./img/gnn_design_3.png)


# Training Overview

![GNN Design Overview 4](./img/gnn_design_4.png)


The same aggregation parameters are shared for all nodes:
- The number of model parameters is sublinear in $|𝑉|$ and we can generalize to unseen nodes!

![GNN Shared parameters](./img/gnn_shared_params.png)


![Inductive Embedding](./img/inductive_node_embedding.png)

- Inductive node embedding Generalize to entirely unseen graphs
- E.g., train on protein interaction graph from model organism A and generate embeddings on newly collected data about organism B

![Inductive Embedding](./img/dynamic_graph_node_embedding.png)
- Many application settings constantly encounter previously unseen nodes:
    - E.g., Reddit, YouTube, Google Scholar
- Need to generate new embeddings on demand

# Summary

- **Recap**: Generate node embeddings by aggregating neighborhood information.
  - We saw a **basic variant** of this idea.
  - Key distinctions are in how different approaches aggregate information across the layers.

- **Next**: Describe GraphSAGE graph neural network architecture.
